In [5]:
import os
import numpy as np
import pandas as pd

In [6]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/compustat_profit.csv' # Fundamental data
df = pd.read_csv(file_path_1)
df['datadate'] = pd.to_datetime(df['datadate'])
df['rdq'] = pd.to_datetime(df['rdq'])
df = df.drop(columns=['gvkey', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df = df.sort_values(by=['tic', 'rdq'])

df.head()

,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq
1243,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0
1244,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0
1245,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0
1246,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0
1247,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0


In [7]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/prices_profit.csv' # price data
df2 = pd.read_csv(file_path_2)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.drop(columns=['PERMNO', 'OPENPRC'])

df2 = df2.sort_values(by=['TICKER', 'date'])
df2 = df2.rename(columns={'TICKER': 'tic'})

df2['PRC'] = df2['PRC'].abs()
#df2['OPENPRC'] = df2['OPENPRC'].abs()

df2['30_day_MA'] = df2.groupby('tic')['PRC'].rolling(window=30, min_periods=1).mean().reset_index(level=0, drop=True)


df2.head()

,date,tic,PRC,30_day_MA
312596,1980-12-12,AAPL,28.8125,28.812500
312597,1980-12-15,AAPL,27.3125,28.062500
312598,1980-12-16,AAPL,25.3125,27.145833
312599,1980-12-17,AAPL,25.9375,26.843750
312600,1980-12-18,AAPL,26.6875,26.812500


In [8]:
import pandas as pd
df = df.dropna(subset=['rdq'])

# Merge the dataframes on 'tic' and ensure that the price is on or after 'rdq'
df = pd.merge_asof(df.sort_values('datadate'), 
                          df2.sort_values('date'), 
                          by='tic', 
                          left_on='datadate', 
                          right_on='date', 
                          direction='forward')

# Now merged_df will have an extra column with the stock price for the date closest to 'rdq'
df = df.sort_values(by=['tic', 'rdq'])
df.head()


,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,30_day_MA
1678,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0,1981-06-30,26.0625,30.854167
1748,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0,1981-09-30,15.3125,18.756250
1784,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0,1981-12-31,22.1875,19.820833
1831,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0,1982-03-31,16.9375,17.093750
1858,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0,1982-06-30,12.8125,13.558333


In [9]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df3.tail()

,caldt,spindx
25794,2023-12-22,4754.63
25795,2023-12-26,4774.75
25796,2023-12-27,4781.58
25797,2023-12-28,4783.35
25798,2023-12-29,4769.83


In [10]:
df = df.sort_values(by=['datadate'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='datadate', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])
df = df[df['date'] <= '2023-12-29']

df['PRC'] = df['PRC'] / df['ajexq']
#df['OPENPRC'] = df['OPENPRC'] / df['ajexq']
df['epsfiq'] = df['epsfiq'] / df['ajexq']
df['cshoq'] = df['cshoq'] * df['ajexq']
df['cshfdy'] = df['cshfdy'] * df['ajexq']

df = df.sort_values(by=['tic', 'rdq']).reset_index(drop=True)
df.tail()

,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,30_day_MA,spindx
15172,2022-09-30,ZTS,1.0,2022-11-03,6551.0,466.442,1.13,4662.0,471.600,1171.0,0.325,2022-09-30,148.28999,157.579000,3585.62
15173,2022-12-31,ZTS,1.0,2023-02-14,7506.0,463.808,0.99,4403.0,470.385,1912.0,0.325,2023-01-03,146.85001,149.229334,3839.50
15174,2023-03-31,ZTS,1.0,2023-05-04,6269.0,462.495,1.19,4491.0,464.600,549.0,0.375,2023-03-31,166.44000,166.095001,4109.31
15175,2023-06-30,ZTS,1.0,2023-08-08,6183.0,460.751,1.45,4621.0,463.800,732.0,0.375,2023-06-30,172.21001,169.091000,4450.38
15176,2023-09-30,ZTS,1.0,2023-11-02,6222.0,459.525,1.29,5074.0,463.000,1456.0,0.375,2023-10-02,172.73000,182.758666,4288.05


In [11]:
# compute ratios
# Use OPENPRC first, if NaN then fall back to PRC

df['ptb'] = (df['30_day_MA'] * df['cshoq']) / df['teqq']
df['div_yield'] = df['dvpspq'] / df['30_day_MA']
df['pe_inc'] = df['epsfiq'] / df['ajexq']
#df['pcf'] = (df['OPENPRC'].fillna(df['PRC']) * df['cshfdy']) / df['oancfy']

df = df.drop(columns=['ajexq', 'date', '30_day_MA', 'cshoq', 'cshfdy', 'oancfy'])


df.tail(10)


,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,div_yield,pe_inc
15167,2021-06-30,ZTS,2021-08-05,7048.0,1.07,4352.0,0.250,186.36000,4297.50,19.645142,0.001387,1.07
15168,2021-09-30,ZTS,2021-11-04,6682.0,1.16,4681.0,0.250,194.14000,4307.54,20.586258,0.001228,1.16
15169,2021-12-31,ZTS,2022-02-15,6930.0,0.87,4544.0,0.250,244.03000,4766.18,24.047194,0.001081,0.87
15170,2022-03-31,ZTS,2022-05-05,6837.0,1.26,4658.0,0.325,188.59000,4530.41,19.336632,0.001700,1.26
15171,2022-06-30,ZTS,2022-08-04,6651.0,1.12,4580.0,0.325,171.89000,3785.38,16.979343,0.001958,1.12
15172,2022-09-30,ZTS,2022-11-03,6551.0,1.13,4662.0,0.325,148.28999,3585.62,15.766080,0.002062,1.13
15173,2022-12-31,ZTS,2023-02-14,7506.0,0.99,4403.0,0.325,146.85001,3839.50,15.719682,0.002178,0.99
15174,2023-03-31,ZTS,2023-05-04,6269.0,1.19,4491.0,0.375,166.44000,4109.31,17.104900,0.002258,1.19
15175,2023-06-30,ZTS,2023-08-08,6183.0,1.45,4621.0,0.375,172.21001,4450.38,16.859738,0.002218,1.45
15176,2023-09-30,ZTS,2023-11-02,6222.0,1.29,5074.0,0.375,172.73000,4288.05,16.551473,0.002052,1.29


In [12]:


# convert features to pct

features = [
    'actq', 'epsfiq', 'teqq',
    'dvpspq', 'ptb', 'div_yield',
    'pe_inc']

for col in features:
    df[col + '_change'] = df[col].pct_change()


# Calculate the percentage change for 'PRC' within each stock group
df['PRC_change'] = df.groupby('tic')['PRC'].pct_change()
df['next_PRC_change'] = df.groupby('tic')['PRC_change'].shift(-1)

df['spindx_change'] = df.groupby('tic')['spindx'].pct_change()
df['next_spindx_change'] = df.groupby('tic')['spindx_change'].shift(-1)

df['relative_quarterly_return'] = (df['PRC_change'] - df['spindx_change'])
df['next_relative_quarterly_return'] = (df['next_PRC_change'] - df['next_spindx_change'])

#df = df.drop(columns=['prccq', 'spindx'])

df.tail()

,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,...,dvpspq_change,ptb_change,div_yield_change,pe_inc_change,PRC_change,next_PRC_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
15172,2022-09-30,ZTS,2022-11-03,6551.0,1.13,4662.0,0.325,148.28999,3585.62,15.766080,...,0.000000,-0.071455,0.053110,0.008929,-0.137297,-0.009711,-0.052771,0.070805,-0.084526,-0.080516
15173,2022-12-31,ZTS,2023-02-14,7506.0,0.99,4403.0,0.325,146.85001,3839.50,15.719682,...,0.000000,-0.002943,0.055952,-0.123894,-0.009711,0.133401,0.070805,0.070272,-0.080516,0.063129
15174,2023-03-31,ZTS,2023-05-04,6269.0,1.19,4491.0,0.375,166.44000,4109.31,17.104900,...,0.153846,0.088120,0.036682,0.202020,0.133401,0.034667,0.070272,0.082999,0.063129,-0.048332
15175,2023-06-30,ZTS,2023-08-08,6183.0,1.45,4621.0,0.375,172.21001,4450.38,16.859738,...,0.000000,-0.014333,-0.017718,0.218487,0.034667,0.003020,0.082999,-0.036476,-0.048332,0.039495
15176,2023-09-30,ZTS,2023-11-02,6222.0,1.29,5074.0,0.375,172.73000,4288.05,16.551473,...,0.000000,-0.018284,-0.074785,-0.110345,0.003020,NaN,-0.036476,NaN,0.039495,NaN


In [13]:
# Replace inf and -inf with NaN
df.replace(['inf', '-inf', np.inf, -np.inf], np.nan, inplace=True)

df.head()

,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,...,dvpspq_change,ptb_change,div_yield_change,pe_inc_change,PRC_change,next_PRC_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
0,1981-06-30,AAPL,1981-07-22,201.262,0.000937,NaN,0.0,0.116350,131.21,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.412470,NaN,-0.114549,NaN,-0.297921
1,1981-09-30,AAPL,1981-10-20,227.104,0.000848,NaN,0.0,0.068359,116.18,NaN,...,NaN,NaN,NaN,-0.095238,-0.412470,0.448980,-0.114549,0.054829,-0.297921,0.394151
2,1981-12-31,AAPL,1982-01-22,249.160,0.001071,NaN,0.0,0.099051,122.55,NaN,...,NaN,NaN,NaN,0.263158,0.448980,-0.236620,0.054829,-0.086414,0.394151,-0.150206
3,1982-03-31,AAPL,1982-04-15,263.140,0.001071,NaN,0.0,0.075614,111.96,NaN,...,NaN,NaN,NaN,0.000000,-0.236620,-0.243542,-0.086414,-0.020990,-0.150206,-0.222553
4,1982-06-30,AAPL,1982-07-16,283.437,0.001161,NaN,0.0,0.057199,109.61,NaN,...,NaN,NaN,NaN,0.083333,-0.243542,0.429268,-0.020990,0.098622,-0.222553,0.330646


In [15]:
df.to_csv('dati_trattati_ptest.csv', index=False)  # Set index=False to exclude the index column